In [101]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [102]:
#Get the data
file_name = pd.read_csv('https://query.data.world/s/gib6aa6n3tmtuvrqpbdlhcbgdg7tk4')

In [103]:
# Best Practice: Make a copy of the raw data to work on
reviews = file_name.copy()

In [104]:
reviews.sample(5)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
549593,583,Saint Louis Brewery / Schlafly Tap Room,1209094753,4.0,4.0,3.5,dyan,Oatmeal Stout,4.0,4.0,Schlafly Coffee Stout,5.7,1994
36162,807,George Bateman and Son Ltd. / Batemans Brewery,1303087744,4.0,3.5,4.0,colts9016,Extra Special / Strong Bitter (ESB),4.0,4.0,Batemans XXXB,4.8,2572
1122245,141,Smuttynose Brewing Company,1272241006,3.5,4.0,4.0,keithmurray,Quadrupel (Quad),3.5,3.5,Smuttynose Gravitation (Big Beer Series),12.0,46970
65547,711,Alley Kat Brewing Company,1307297021,3.5,3.5,3.0,MAB,Maibock / Helles Bock,3.0,3.5,Loaded Goat Maibock,7.3,68497
1411122,664,Wells & Young's Ltd,1212708769,4.0,3.5,3.5,atigerlife,Extra Special / Strong Bitter (ESB),4.0,3.5,Young's Special London Ale,6.4,724


In [105]:
reviews = reviews[(reviews['review_overall'] >= 1)|\
       (reviews['review_appearance'] >= 1)]

In [106]:
# Filter 'region' to Total U.S. rows
reviews = reviews[reviews.brewery_id == 147]

In [107]:
reviews['Score'] = ((reviews['review_appearance'] + reviews['review_aroma'] + reviews['review_palate'] + reviews['review_taste'])*5)

In [108]:
reviews.sample(5)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,Score
893648,147,Stone Brewing Co.,1166413609,5.0,4.5,4.5,vwcraig,American Strong Ale,4.0,4.5,Oaked Arrogant Bastard Ale,7.2,8951,87.5
888442,147,Stone Brewing Co.,1318817979,3.5,4.0,3.5,kennylopez,Milk / Sweet Stout,3.0,4.0,Jason Fields & Kevin Sheppard / Tröegs / Stone...,7.3,71405,72.5
899956,147,Stone Brewing Co.,1251054929,5.0,3.5,4.5,MaxBrews,American Pale Ale (APA),4.0,3.5,Stone Pale Ale,5.4,87,77.5
877100,147,Stone Brewing Co.,1216243485,4.0,4.0,4.0,cbros,American IPA,4.0,4.5,Stone IPA (India Pale Ale),6.9,88,82.5
889674,147,Stone Brewing Co.,1316987541,4.5,4.0,4.5,movingtrain,American Black Ale,4.5,4.5,Stone 15th Anniversary Escondidian Imperial Bl...,10.8,72138,87.5


In [109]:
reviews = reviews.drop(['brewery_name','review_time','review_overall', 'review_aroma','review_palate', 'review_appearance','review_profilename','beer_name', 'brewery_id'], axis=1)

In [110]:
X=reviews.drop('beer_style',axis=1) #Predictors
y=reviews['beer_style'] #Response
X.head()

,review_taste,beer_abv,beer_beerid,Score
854097,4.5,8.7,38470,87.5
854115,4.0,8.7,38470,85.0
854414,4.5,7.2,92,90.0
854660,4.5,7.2,92,85.0
854680,2.0,7.2,92,50.0


In [111]:
from sklearn.preprocessing import LabelEncoder
Encoder_X = LabelEncoder() 
for col in X.columns:
    X[col] = Encoder_X.fit_transform(X[col])
Encoder_y=LabelEncoder()
y = Encoder_y.fit_transform(y)

In [112]:
X.head()

,review_taste,beer_abv,beer_beerid,Score
854097,7,25,44,24
854115,6,25,44,23
854414,7,12,3,25
854660,7,12,3,23
854680,2,12,3,9


In [113]:
X=pd.get_dummies(X,columns=X.columns,drop_first=True)
X.head()

,review_taste_1,review_taste_2,review_taste_3,review_taste_4,review_taste_5,review_taste_6,review_taste_7,review_taste_8,beer_abv_1,beer_abv_2,...,Score_20,Score_21,Score_22,Score_23,Score_24,Score_25,Score_26,Score_27,Score_28,Score_29
854097,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
854115,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
854414,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
854660,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
854680,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
#Test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [115]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [116]:
# Train the model using the training data
LogReg = LogisticRegression(solver='sag')
LogReg.fit(X_train, y_train)

C:\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


LogisticRegression(solver='sag')

In [118]:
LogReg.score(X_test, y_test)

0.9995967741935484

In [119]:
from sklearn import svm

In [120]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X, y)

DecisionTreeRegressor()

In [121]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, X, y,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [122]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)

Scores: [1.80095059 7.08708626 0.23003924 1.46628267 2.77864986 0.64269993
 5.0651831  0.58644705 3.16853672 1.2384982 ]
Mean: 2.4064373637020977
Standard deviation: 2.0849741189692237


In [125]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8]},
    {'bootstrap': [False], 'n_estimators':[3,10], 'max_features':[2,3,4]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid,cv=5,
                          scoring='neg_mean_squared_error',
                          return_train_score=True)

grid_search.fit(X,y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')